In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import random
import re
#import ktrain
#from ktrain import text
from joblib import dump, load

In [7]:
df = pd.read_csv('data/Hackathon_Base_Treino_comdep.csv')
dfsub = pd.read_csv('data/Hackathon_Base_Teste.csv')

In [8]:
less = (df['SUB-CATEGORIA'].value_counts() < 2)
cats = []
for i in range(len(less)):
    if(less[i]):
        cats.append(less.keys()[i])
for i in range(len(df)):
    if(df['SUB-CATEGORIA'][i] in cats):
        df = df.append(df.iloc[i,:])

0


In [9]:
sub_cat_enc = LabelEncoder()
target_catsub = sub_cat_enc.fit_transform(df['SUB-CATEGORIA'])
x_train, x_test, y_train, y_test = train_test_split(df['DESCRIÇÃO PARCEIRO'], target_catsub, test_size = 0.1, stratify = target_catsub, random_state=42)

vect = TfidfVectorizer(max_features=10000).fit(x_train)

In [ ]:
t2 = text.Transformer('distilroberta-base', maxlen=100, classes = np.unique(target_catsub))
train2 = t2.preprocess_train(list(x_train), y_train)
test2 = t2.preprocess_test(list(x_test), y_test)
model2 = t2.get_classifier()
learner2 = ktrain.get_learner(model2, train_data = train2, val_data=test2, batch_size=6)
learner2.model.load_weights('models/subcat/weights-20.hdf5')

t = text.Transformer('neuralmind/bert-base-portuguese-cased', maxlen=100, classes = np.unique(target_catsub))
train = t.preprocess_train(list(x_train), y_train)
test = t.preprocess_test(list(x_test), y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data = train, val_data=test, batch_size=6)
learner.model.load_weights('models/subcat/weights-21.hdf5')

In [16]:
nb = load('models/subcat/nb.joblib') 
mlp = load('models/subcat/mlp.joblib') 
cnb = load('models/subcat/cnb.joblib') 

In [21]:
mlp

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [ ]:
model_weights = {'BERT': 0.8365236754871361,
 'BERT2': 0.5430888111169458,
 'CNB': 0.051903171320763775,
 'MLP': 0.5671334042175022,
 'MULTINOMIALNB': 0.0020652658573350998}

model_dict = dict(zip(['BERT', 'MULTINOMIALNB', "CNB", "BERT2", 'MLP'],
                      [learner, nb, cnb, learner2, mlp]))

In [ ]:
aux = dfsub['DESCRIÇÃO PARCEIRO']

## Bert1
predictor = ktrain.get_predictor(learner.model, preproc = t)
pred = predictor.predict(list(aux), return_proba = True)
pred = pred*model_weights['BERT']

## Bert2
predictor = ktrain.get_predictor(learner2.model, preproc = t2)
pred3 = predictor.predict(list(aux), return_proba = True)
pred3 = pred3*model_weights['BERT2']

## Others models
pred2 = 0
for model_name, model in model_dict.items():
    print(model_name)
    if((model_name != "BERT") & (model_name != "BERT2")):
        pred2 += model_dict[model_name].predict_proba(vect.transform(aux)) * model_weights[model_name]
        
## Average
pred6 = (pred2+pred+pred3)/sum(model_weights.values())
dfsub['SUB-CATEGORIA'] = sub_cat_enc.inverse_transform(np.argmax(pred6, axis=1))
dfsub.to_csv('data/Hackathon_Base_Teste.csv', index = False)